<a href="https://colab.research.google.com/github/adamclark/jupyter-notebooks/blob/main/RL/delayed-reward/RL_delayed_reward_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stable-baselines3 -q

In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
# 🛠️ Custom environment with only terminal reward
class TerminalRewardEnv(gym.Env):
    def __init__(self):
        super(TerminalRewardEnv, self).__init__()
        self.observation_space = spaces.Box(low=0, high=1, shape=(4,), dtype=np.float32)
        self.action_space = spaces.Discrete(2)
        self.current_step = 0
        self.max_steps = 20

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        return self._get_obs(), {}

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= self.max_steps
        truncated = False  # Use True if episode truncated by timeout or external reason

        reward = 0.0
        if done:
            # Reward is only given at the end of the episode
            reward = np.random.choice([1.0, 0.0], p=[0.3, 0.7])  # Random success
        return self._get_obs(), reward, done, truncated, {}

    def _get_obs(self):
        return np.random.rand(4).astype(np.float32)

# 📋 Callback to print final reward of each episode
class TerminalRewardLogger(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.episode_rewards = []
        self.episode_step_count = 0

    def _on_step(self) -> bool:
        self.episode_step_count += 1
        done = self.locals["dones"][0]
        reward = self.locals["rewards"][0]
        if done:
            self.episode_rewards.append(reward)
            print(f"Episode {len(self.episode_rewards)} took {self.episode_step_count} steps and ended with reward: {reward}")
            self.episode_step_count = 0
        return True

In [ ]:
# 🎮 Instantiate env and model
env = TerminalRewardEnv()
model = PPO("MlpPolicy", env, verbose=0)

# 🚀 Train with reward logger
logger = TerminalRewardLogger()
model.learn(total_timesteps=5000, callback=logger)

Episode 1 took 20 steps and ended with reward: 0.0
Episode 2 took 20 steps and ended with reward: 0.0
Episode 3 took 20 steps and ended with reward: 0.0
Episode 4 took 20 steps and ended with reward: 0.0
Episode 5 took 20 steps and ended with reward: 0.0
Episode 6 took 20 steps and ended with reward: 1.0
Episode 7 took 20 steps and ended with reward: 1.0
Episode 8 took 20 steps and ended with reward: 0.0
Episode 9 took 20 steps and ended with reward: 0.0
Episode 10 took 20 steps and ended with reward: 0.0
Episode 11 took 20 steps and ended with reward: 0.0
Episode 12 took 20 steps and ended with reward: 1.0
Episode 13 took 20 steps and ended with reward: 0.0
Episode 14 took 20 steps and ended with reward: 1.0
Episode 15 took 20 steps and ended with reward: 0.0
Episode 16 took 20 steps and ended with reward: 0.0
Episode 17 took 20 steps and ended with reward: 0.0
Episode 18 took 20 steps and ended with reward: 0.0
Episode 19 took 20 steps and ended with reward: 1.0
Episode 20 took 20 st